# Модель классификации тональности текста

ИКлиенту нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. В нашем распоряжении размеченные комментарии на английском языке.

Цель проекта - обучить модель классифицировать комментарии на позитивные и негативные, метрика качества *F1* должна быть не меньше 0.75. 

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Градиентный-бустинг-LGBM" data-toc-modified-id="Градиентный-бустинг-LGBM-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Градиентный бустинг LGBM</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Подготовка

In [25]:
import pandas as pd
import numpy as np
import re

#sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# gradient boost
import lightgbm as lgb

#nltk
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

Загрузим и ознакомимся с данными:

In [3]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
print('Найдено дубликатов:', df.duplicated().sum())

Найдено дубликатов: 0


В вашем распоряжении набор данных с разметкой о токсичности правок. Текст комментариев на английском языке. Дубликатов в датасете не обнаружено. На этапе подготовки нам предстоит очистить текст от знаков препинания, привести к нижнему регистру, разбить на слова и провести лемматизацию.

Загрузим необходимые пакеты из библиотеки nltk:

In [6]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vshevchenko\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vshevchenko\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vshevchenko\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vshevchenko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Избавимся от наиболее популярных сокращений с помощью словаря сокращений:

In [7]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and ",
"won't": "would not",
'dis': 'this',
'bak': 'back',
'brng': 'bring'}

Напишем функцию, очищающую текст от сокращений. На основе словаря функция заменит сокращения на полный вариант слов и выражений. Передадим функции столбец датасета с текстом комментариев:

In [8]:
def del_contractions(x):
    if type(x) is str:
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        return x
    else:
        return x

In [9]:
df['text'] = df['text'].apply(del_contractions)

Напишем функцию для разметки текста - присвоим словам теги, обозначающие часть речи. Далее напишем функцию для лемматизации нашего корпуса текста: функция приведёт текст к нижнему регистру, очистит текст от знаков препинания и лишних пробелов, проведёт токенизацию.

In [10]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [11]:
def lem(df): 
    text = df.lower()
    text =  " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split())
    word_list = nltk.word_tokenize(text)
    lemmatized = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list])
    return lemmatized                

Лемматизированный текст сохраним в отдельном столбце датафрейма:

In [12]:
df['text_lemmatized'] = df['text'].apply(lem)

In [13]:
df

,text,toxic,text_lemmatized
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI cannot make any real suggestions on...",0,more i can not make any real suggestion on imp...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,and for the second time of ask when your view ...
159567,You should be ashamed of yourself \n\nThat is ...,0,you should be ashamed of yourself that be a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm there no actual article for prosti...
159569,And it looks like it was actually you who put ...,0,and it look like it be actually you who put on...


Проверим, не образовалось ли после лемматизации пропусков:

In [14]:
df.isna().sum()

text               0
toxic              0
text_lemmatized    0
dtype: int64

In [15]:
df.shape

(159571, 3)

Объявим переменную corpus, которой передадим значения из столбца с лемматизированными комментариями:

In [28]:
corpus = df['text_lemmatized'].values
corpus

array(['explanation why the edits make under my username hardcore metallica fan be revert they weren t vandalism just closure on some gas after i vote at new york doll fac and please do not remove the template from the talk page since i m retire now',
       'd aww he match this background colour i m seemingly stuck with thanks talk january utc',
       'hey man i m really not try to edit war it s just that this guy be constantly remove relevant information and talk to me through edits instead of my talk page he seem to care more about the format than the actual info',
       ...,
       'spitzer umm there no actual article for prostitution ring crunch captain',
       'and it look like it be actually you who put on the speedy to have the first version delete now that i look at it',
       'and i really do not think you understand i come here and my idea be bad right away what kind of community go you have bad idea go away instead of help rewrite them'],
      dtype=object)

Разделим выборки на обучающую и тестовую:

In [29]:
features_train, features_test, target_train, target_test = train_test_split(
    corpus, df['toxic'], test_size = 0.2, random_state = 12345)

In [30]:
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(127656,)
(127656,)
(31915,)
(31915,)


Мы готовы приступить к обучению моделей.

## Обучение

### Логистическая регрессия

Создадим пайплайн для логистической регрессии: векторизируем тексты и подберём гиперпараметры с помощью GridSearch:

In [31]:
stop_words = stopwords.words('english')

pipe_logreg = Pipeline([('vect', TfidfVectorizer(stop_words = stop_words)), 
                        ('logistic_regression', LogisticRegression(random_state = 12345))])

param_grid_logreg = [{
    'logistic_regression__penalty' : ['l1', 'l2'],
    'logistic_regression__C' : [0.1, 1, 10],
    'logistic_regression__solver' : ['liblinear'],
    'logistic_regression__class_weight' : [None, 'balanced']}]

In [33]:
print('Наилучшие параметры:', logreg_gs.best_estimator_)

Наилучшие параметры: Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('logistic_regression',
                 LogisticRegression(C=1, penalty='l1', random_state=12345,
                                    solver='liblinear'))])
Среднее значение F1 логистической регрессии после кросс-валидации: 0.77


In [39]:
print('Среднее значение F1 логистической регрессии после кросс-валидации: {:.2f}'.format(logreg_gs.best_score_))

Среднее значение F1 логистической регрессии после кросс-валидации: 0.77


### Случайный лес

Аналогичным образом запустим процесс векторизации и подбора гиперпараметров для модели случайного леса. Воспользуемся RandomizedSearch:

In [36]:
pipe_forest = Pipeline([('vect', TfidfVectorizer(stop_words = stop_words)), 
                        ('forest', RandomForestClassifier(random_state = 12345))])

param_grid_forest = [{
    'forest__n_estimators': [50, 200, 500],
    'forest__max_depth': [10, 30, 80, 100],
    'forest__min_samples_split': [2, 5, 10],
    'forest__class_weight' : [None, 'balanced']}]

In [38]:
forest = RandomizedSearchCV(estimator = pipe_forest,
                            param_distributions = param_grid_forest,
                            scoring='f1', cv = 5, n_iter = 10)
forest.fit(features_train, target_train)
print('Наилучшие параметры:', forest.best_estimator_)

Наилучшие параметры: Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('forest',
                 RandomForestClassifier(class_weight='balanced', max_depth=80,
                                        min_samples_split=5, n_estimators=50,
                                        random_state=12345))])
Среднее значение F1 модели random forest после кросс-валидации: 0.52


In [40]:
print('Среднее значение F1 модели random forest после кросс-валидации: {:.2f}'.format(forest.best_score_))

Среднее значение F1 модели random forest после кросс-валидации: 0.52


### Градиентный бустинг LGBM

Попробуем модель градиентного бустинга. Подберём параметры для модели LGBM:

In [43]:
pipe_lgbm = Pipeline([('vect', TfidfVectorizer(stop_words = stop_words)), 
                      ('lgbm', lgb.LGBMClassifier(random_state=12345))])

param_grid_lgbm = [{
    'lgbm__num_leaves': [50, 250, 500],
    'lgbm__learning_rate': [0.2, 0.03],
    'lgbm__max_depth': [10, 30, 80, 100],
    'lgbm__n_estimators': [50, 200, 500],
    'lgbm__class_weight' : [None, 'balanced']
}]

print('Запуск RandomizedSearch...')
lgbm_model = RandomizedSearchCV(estimator = pipe_lgbm,
                                param_distributions = param_grid_lgbm,
                                scoring='f1', cv = 5, n_iter = 20)
lgbm_model.fit(features_train, target_train)
print('Наилучшие параметры:', lgbm_model.best_estimator_)

Запуск RandomizedSearch...
Наилучшие параметры: Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('lgbm',
                 LGBMClassifier(class_weight='balanced', learning_rate=0.03,
                                max_depth=100, n_estimators=500, num_leaves=500,
                                random_state=12345))])


In [44]:
print('Среднее значение F1 модели LGBM после кросс-валидации: {:.2f}'.format(lgbm_model.best_score_))

Среднее значение F1 модели LGBM после кросс-валидации: 0.77


Представим результаты наших моделей в таблице:

In [46]:
data = [
    ['Модель градиентного бустинга LightGBM', '{:.2f}'.format(lgbm_model.best_score_)],
    ['Модель логистической регрессии', '{:.2f}'.format(logreg_gs.best_score_)],
    ['Модель случайного леса', '{:.2f}'.format(forest.best_score_)],
]
columns = ['Модель', 'Значение F1']
results = pd.DataFrame(data=data, columns=columns)
results

,Модель,Значение F1
0,Модель градиентного бустинга LightGBM,0.77
1,Модель логистической регрессии,0.77
2,Модель случайного леса,0.52


## Тестирование

Проверим модели на тестовой выборке:

In [48]:
print('Значение F1 модели random forest на тестовой выборке: {:.2f}'.format(
    f1_score(target_test, forest.predict(features_test))))

Значение F1 модели random forest на тестовой выборке: 0.50


In [50]:
print('Значение F1 модели LGBM на тестовой выборке: {:.2f}'.format(
    f1_score(target_test, lgbm_model.predict(features_test))))

Значение F1 модели LGBM на тестовой выборке: 0.77


In [51]:
print('Значение F1 модели логистической регрессии на тестовой выборке: {:.2f}'.format(
    f1_score(target_test, logreg_gs.predict(features_test))))

Значение F1 модели логистической регрессии на тестовой выборке: 0.77


На тестовой выборке модели градиентного бустинга и логистической регрессии показали результаты, аналогичные значениям после валидации.

Протестируем наши модели на вменяемость, сравнив полученные значения метрик с тем, что покажет константная модель.

In [66]:
dummy_class = DummyClassifier(strategy='constant', constant=1)
dummy_class.fit(features_train, target_train)
print('Значение F1 константной модели: {:.2f}'.format(
    f1_score(target_test, dummy_class.predict(features_test))))

Значение F1 константной модели: 0.18


Все модели прошли проверку на вменяемость.

## Выводы

Наилучший результат показала **модель линейной регрессии**. Значение метрики F1 на тестовой выборке равно тому, что было получено после валидации -- **0,77**, качество предсказания не ухудшилось, при этом время обучения линейной регрессии наименьшее. Именно эта модель рекомендуется к использованию.